In [ ]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import chess
import random
import inspect
from datetime import datetime
import multiprocessing as mp
import chess.polyglot
import chess.pgn
import numpy as np

# Utils

In [ ]:
def generate_game(player1,name1,ply1,player2,name2,ply2,game_num):
    board = chess.Board()

    book_paths = ["/content/drive/MyDrive/aberturas/baron30.bin","/content/drive/MyDrive/aberturas/gm2001.bin","/content/drive/MyDrive/aberturas/komodo.bin","/content/drive/MyDrive/aberturas/rodent.bin"]
    book = chess.polyglot.open_reader(random.choice(book_paths))

    game = chess.pgn.Game()
    game.headers["Event"] = "Prodigy_tets"
    game.headers["Round"] = game_num
    game.headers["Date"] = datetime.today()
    game.headers["White"] = name1
    game.headers["Black"] = name2
    for i in range(1000):
        openings = []
        for entry in book.find_all(board):
            openings.append(entry.move)

        if len(openings) != 0:
            move = random.choice(openings)

        elif board.turn == chess.WHITE:
            if inspect.isclass(player1):
                move = player1(board,ply1)
                move = move.movement(ply1)
            else:
                move = player1(board,ply1)
            if move == None:
                return game

        else:
            if inspect.isclass(player2):
                move = player2(board,ply2)
                move = move.movement(ply2)
            else:
                move = player2(board,ply2)
            if move == None:
                return game
        board.push(move)
        if i == 0:
            node = game.add_variation(move)
        else:
            node = node.add_variation(move)
        if board.is_checkmate():
            result = not board.turn
            if result == True:
                game.headers["Result"] = "1-0"
            else:
                game.headers["Result"] = "0-1"
            return game
        elif board.is_stalemate():
                game.headers["Result"] = "1/2-1/2"
                return game
        elif board.is_fivefold_repetition():
                game.headers["Result"] = "1/2-1/2"
                return game
        elif board.is_insufficient_material():
                game.headers["Result"] = "1/2-1/2"
                return game
        elif board.can_claim_draw():
                game.headers["Result"] = "1/2-1/2"
                return game

In [ ]:
def create_pgn_file(file_name,num_games,player1,name1,ply1,player2,name2,ply2):
    all_games = []
    for games in range(num_games):
        game = generate_game(player1,name1,ply1,player2,name2,ply2)
        all_games.append(game)
    pgn_strings = [str(game) for game in all_games]
    unified_pgn = "\n\n".join(pgn_strings)

    with open(file_name, "w") as pgn_file:
        pgn_file.write(unified_pgn)

# Competitors

## hawks

In [ ]:
def custom_loss(output, target):
    from_loss = tf.keras.losses.CategoricalCrossentropy()
    to_loss = tf.keras.losses.CategoricalCrossentropy()
    loss = from_loss(output[:,0,:], target[:,0,:]) + to_loss(output[:,1,:], target[:,1,:])
    return loss
import tensorflow as tf
custom_objects = {'custom_loss': custom_loss}
#model = tf.keras.models.load_model('/content/drive/MyDrive/Tcc/hawks_v3.h5',custom_objects=custom_objects)
alphabet_to_number = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7
}

number_to_alphabet = {v: k for k, v in alphabet_to_number.items()}

import numpy as np
def fen_to_numeric(fen_string,piece):
    piece_placement = fen_string.split()[0]
    board = [[0] * 8 for _ in range(8)]
    row_index = 0
    col_index = 0

    for char in piece_placement:
        if char == '/':
            row_index += 1
            col_index = 0
        elif char.isdigit():
            col_index += int(char)
        elif char.lower() == piece.lower():
            piece_value = 1
            if char.islower():
                piece_value *= -1
            board[row_index][col_index] = piece_value
            col_index += 1
        else:
          board[row_index][col_index] = 0
          col_index += 1

    return board

def matrix_board(board):
  pieces = ["p","k","r","n","q","b"]
  layers = []
  for piece in pieces:
    layers.append(fen_to_numeric(board.fen(),piece))
  return np.stack(layers)

def move_matrix(move):
  uci_move = move

  from_output_layer = np.zeros((8,8))
  from_row= 8 - int(uci_move[1])
  from_column= alphabet_to_number[uci_move[0]]
  from_output_layer[from_row, from_column] = 1

  to_output_layer = np.zeros((8,8))
  to_row = 8 - int(uci_move[3])
  tow_column = alphabet_to_number[uci_move[2]]
  to_output_layer[to_row, tow_column] = 1
  return np.stack([from_output_layer,to_output_layer])

def distribution(probabilities):
  dist = np.exp(np.array(probabilities))
  dist = (dist/dist.sum())**3
  return dist/dist.sum()

def hawks(board,ply):
  previous_position = matrix_board(board)
  matrix = np.expand_dims(previous_position, axis=0)

  if board.turn == False:
    matrix *= -1

  predictions = model.predict(matrix)

  y = predictions[0,:,:]

  vals = []
  froms =  [str(legal_move)[:2] for legal_move in board.legal_moves]
  froms = list(set(froms))
  for from_ in froms:
    val = y[0,:,:][8 - int(from_[1]), alphabet_to_number[from_[0]]]
    vals.append(val)

  probs = distribution(vals)
  choosen = str(np.random.choice(froms, size=1, p=probs)[0])[:2]

  values = []
  moves = []
  for move in list(board.legal_moves):
    if choosen == str(move)[:2]:
      to = str(move)[2:]
      val = y[1,:,:][8 - int(to[1]), alphabet_to_number[to[0]]]
      values.append(val)
      moves.append(choosen+to)
  choice = moves[np.argmax(values)]
  return chess.Move.from_uci(choice)

## Hawks v4

In [ ]:
model_v4 = tf.keras.models.load_model('/content/drive/MyDrive/Tcc/hawks_v4.h5',custom_objects=custom_objects)
def hawks_v4(board,ply):
  previous_position = matrix_board(board)
  matrix = np.expand_dims(previous_position, axis=0)

  if board.turn == False:
    matrix *= -1

  predictions = model_v4.predict(matrix)

  y = predictions[0,:,:]

  vals = []
  froms =  [str(legal_move)[:2] for legal_move in board.legal_moves]
  froms = list(set(froms))
  for from_ in froms:
    val = y[0,:,:][8 - int(from_[1]), alphabet_to_number[from_[0]]]
    vals.append(val)

  probs = distribution(vals)
  choosen = str(np.random.choice(froms, size=1, p=probs)[0])[:2]

  values = []
  moves = []
  for move in list(board.legal_moves):
    if choosen == str(move)[:2]:
      to = str(move)[2:]
      val = y[1,:,:][8 - int(to[1]), alphabet_to_number[to[0]]]
      values.append(val)
      moves.append(choosen+to)
  choice = moves[np.argmax(values)]
  return chess.Move.from_uci(choice)

## NoisyBoy

In [ ]:
import tensorflow as tf
model_noisy_boy = tf.keras.models.load_model("/content/drive/MyDrive/models/noisy_boy_v3_no_overfitting.h5")

In [ ]:
def fen_to_numeric(fen_string,piece):
    piece_placement = fen_string.split()[0]
    board = [[0] * 8 for _ in range(8)]
    row_index = 0
    col_index = 0

    for char in piece_placement:
        if char == '/':
            row_index += 1
            col_index = 0
        elif char.isdigit():
            col_index += int(char)
        elif char.lower() == piece.lower():
            piece_value = 1
            if char.islower():
                piece_value *= -1
            board[row_index][col_index] = piece_value
            col_index += 1
        else:
          board[row_index][col_index] = 0
          col_index += 1

    return board

def matrix_board(board):
  pieces = ["p","k","r","n","q","b"]
  layers = []
  for piece in pieces:
    layers.append(fen_to_numeric(board.fen(),piece))
  return np.stack(layers)

class noisy_boy_v1():
  def __init__(self, board, ply):
    self.board = board
    self.ply = ply
    self.cor = self.board.turn

  def avaliacao(self,cor):
    if self.board.is_repetition(2):
      return 0
    if cor:
      return model_noisy_boy.predict(np.array(matrix_board(self.board)).reshape(-1, 6, 8,8),verbose=0)[0].tolist()[0]
    else:
      return -model_noisy_boy.predict(np.array(matrix_board(self.board)).reshape(-1, 6, 8,8),verbose=0)[0].tolist()[0]


  def negamax(self, alpha, beta, ply):
    if self.board.is_checkmate():
        if self.board.turn == self.cor:
          return +9999
        else:
          return -9999
    if ply == 0:
      return self.quisce(-beta,-alpha,3)

    score = 0
    best_value = -1000
    #sorted_moves = sorted(list(self.board.legal_moves), key=is_check_move, reverse=True)
    #print(list(self.board.legal_moves))
    for move in list(self.board.legal_moves):
      self.board.push(move)
      score = -self.negamax(-beta, -alpha, ply-1)
      # print("------- "+str(ply)+ "------")
      # print(str(move) + " : " + str(score))
      self.board.pop()

      if best_value < score:
        best_value = score

      if score >= beta:
        # print("-------Eureka Cutoff------")
        # print(str(move) + " : " + str(score) + " : " + str(beta))
        return beta

      alpha = max(score, alpha)

    return best_value

  def quisce(self,alpha,beta,ply):
    if ply == 0:
      return self.avaliacao(self.board.turn)

    stand_pat = self.avaliacao(self.board.turn)
    if stand_pat >= beta:
      return beta

    delta = 1000

    if alpha < stand_pat:
      alpha = stand_pat

    for move in list(self.board.legal_moves):
      if self.board.is_capture(move):
        self.board.push(move)
        score = -self.quisce(-beta, -alpha, ply-1)
        self.board.pop()

        if move.promotion:
            delta+=750
        if stand_pat < alpha-delta:
            return alpha
        if score >= beta:
            return beta
        if score > alpha:
            alpha = score
    return alpha

  def movement(self, ply):
    best_move = None
    best_value = -1000

    alpha = -1000
    beta = 1000

    for move in list(self.board.legal_moves):
      self.board.push(move)
      score = -self.negamax(-beta, -alpha, ply)
      self.board.pop()

      if best_move == None:
          best_move = move
      if best_value < score:
          best_value = score
          best_move = move

    return best_move

#Show Time

In [ ]:
def create_pgn_file(file_name, num_games, player1, name1, ply1, player2, name2, ply2):
    for game_num in range(1, num_games + 1):
        game = generate_game(player1, name1, ply1, player2, name2, ply2, game_num)
        pgn_string = str(game)

        with open(file_name, "a") as pgn_file:
            if game_num > 1:
                # If this is not the first game, add an extra line for separation
                pgn_file.write("\n\n")
            pgn_file.write(pgn_string)
        print("Game ended: ",game_num)

In [ ]:
def dumb_player(board,ply):
  return random.choice(list(board.legal_moves))

In [ ]:
file_name = "/content/drive/MyDrive/acw2023/Noisy_new_vs_prodigy.pgn"
num_games = 2

name1 = "NoisyBoy"
ply1 = 0
name2 = "Prodigy"
ply2 = 4

create_pgn_file(file_name, num_games, noisy_boy_v1, name1, ply1, engine_v2 , name2, ply2)

Game ended:  1
Game ended:  2
